In [61]:
import category_encoders
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.base import TransformerMixin
from sklearn import preprocessing
from sklearn import neighbors
from sklearn.pipeline import Pipeline
from sklearn import set_config
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
import category_encoders as ce
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler
pd.options.display.max_rows = 20 #Para controlar cuantas filas de los dataframes mostrar.
import warnings
warnings.filterwarnings('ignore') #Para ignorar (o no) las advertencias.

**AQUÍ SOLO SE MUESTRAN TRATAMIENTOS QUE HAY QUE HACERLE A LOS DATOS CON SUS RESPECTIVOS RESULTADOS. NO OBSTANTE, LUEGO HAY QUE METER ESTO EN UN COLUMN TRANSFORMER Y DESPUÉS EN UNA PIPELINE.**

In [ ]:
#Primero, leemos el archivo csv. Está separado por puntos y comas.

datos = pd.read_csv('pokemon-data.csv',delimiter =';')
y = datos.loc[:,'Tier'].copy()
X = datos.loc[:,datos.columns != 'Tier'].copy()
X

In [63]:
#Vamos a modificar el índice de las filas
X.index = X['Name']

Esto es un ejemplo.

In [64]:
class TierCathegorizer(TransformerMixin):
    def transform(self,X,y=None):
        X = pd.DataFrame(X)
        yaux = X.copy()
        tiers = np.where(np.logical_or(yaux.loc[:,'Tier'] == 'OU', yaux.loc[:,'Tier'] == 'Uber', yaux.loc[:,'Tier'] == 'AG'),'bueno','malo')
        yaux.loc[:,'Tier'] = tiers
        return yaux


In [ ]:
#Primero, leemos el archivo csv. Está separado por puntos y comas.

datos = pd.read_csv('pokemon-data.csv',delimiter =';')
y = datos.loc[:,'Tier'].copy()
X = datos.loc[:,datos.columns != 'Tier'].copy()
X

In [66]:
#Vamos a modificar el índice de las filas
X.index = X['Name']

Para reducir el numero de clases para la variable de salida, juntamos los Pokemons en "Buenos" y "Malos". La categoría de "buenos" va a ser nuestra categoría "positiva", ya que es la que menos ejemplos posee, pero a su vez es la más interesante, ya que los jugadores competitivos quieren saber qué Pokémon son los mejores.

In [67]:
class TierCathegorizer(TransformerMixin):
    def transform(self,X,y=None):
        X = pd.DataFrame(X)
        yaux = X.copy()
        tiers = np.where(np.logical_or(yaux.loc[:,'Tier'] == 'OU', yaux.loc[:,'Tier'] == 'Uber', yaux.loc[:,'Tier'] == 'AG'),'bueno','malo')
        yaux.loc[:,'Tier'] = tiers
        return yaux

In [ ]:
pipe = Pipeline([('tier',TierCathegorizer())])
y_transformed = pipe.transform(y)
print("Malos: "+str(y_transformed[y_transformed=="malo"].count()[0]))
print("Buenos: "+str(y_transformed[y_transformed=="bueno"].count()[0]))

Juntamos los movimientos en una nueva columna que está compuesta por el número de movimientos. Sin embargo, el Pokemon Smeargle es una excepción porque puede aprender todos los movimientos, pero esto no está reflejado en la base de datos (Aparece con sólo 10 movimientos).

In [69]:
class MovementCounter(TransformerMixin):
    def transform(self, X, y=None):
        X = pd.DataFrame(X)
        Xaux = X.copy()
        Xaux['Moves'] = Xaux['Moves'].astype(str).str.count(',') + 1
        Xaux.loc['Smeargle','Moves'] = 676
        return Xaux


In [ ]:
pipe = Pipeline([('Movement', MovementCounter())])
X_Count = pipe.transform(X)
X_Count

Nos interesa desdoblar las columnas de los tipos, ya que hay muchas combinaciones de tipos diferentes. Al desdoblar la columna de tipos, obtenemos dos variables que sólo pueden tener 18 categorías (tipo 2 tendrá 19 categorías ya que a los Pokémon que sólo tienen un tipo les asignaremos un segundo tipo "nulo"), lo cual nos va a dar más información sobre lo bueno que es cada tipo a la hora
de  determinar el potencial competitivo de un Pokémon. Para ello, creamos una clase `TypeCategorizer()`que se va a encargar de esto. Tiene los métodos `fit` y `transform`para poder utilziarse en una *Pipeline*.

In [71]:
class TypeCategorizer(TransformerMixin):
    # Constructor de la clase
    def __init__(self, columns=None): #Si recibimos algo
        self.columns = columns

    #Fit
    def fit(self, X):
        X=pd.DataFrame(X) #Por si recibiéramos un array de numpy
        if self.columns == None:
            self.columns=X.columns
        return self

    #Transform
    def transform(self, X):
        #Creamos una copia de X para no perder los datos originales
        Xaux=X.copy()
        #Accedemos a la columna de tipos, y rellenamos las columnas acorde
        for c in self.columns:
            caracteresAEliminar = ["[", "]", "'"]
            for char in caracteresAEliminar:
                Xaux.loc[:,c]= Xaux.loc[:,c].str.replace(char,'')
            #Después, para los Pokémon que solo tienen un tipo, vamos a añadirles un segundo tipo "nulo"
            Xaux[['Type 1', 'Type 2']] = Xaux.loc[:,c].str.split(',', expand=True) #Expand true para que cree dos nuevas columnas

            #Por último, hay que borrar la columna de "Types" original
            Xaux=Xaux.drop(["Types"], axis=1)
        return Xaux

In [ ]:
categorizadorDeTipos = TypeCategorizer(columns = ["Types"])
categorizadorDeTipos.fit(X)
X_ConTipos = categorizadorDeTipos.transform(X_Count)
X_ConTipos

Vamos a realizar el mismo tratamiento para las habilidades. Es decir, vamos a generar tres columnas, una por cada habilidad (un Pokémon puede tener 1,2 ó 3 habilidades de las que escoger).

In [73]:
def countCaracter(text, character):
    apariciones=0
    for char in text:
        if char==character:
            apariciones+=1
    return apariciones

class AbilityCategorizer(TransformerMixin):
    # Constructor de la clase
    def __init__(self, columns=None): #Si recibimos algo
        self.columns = columns

    #Fit
    def fit(self, X):
        X=pd.DataFrame(X) #Por si recibiéramos un array de numpy
        if self.columns == None:
            self.columns=X.columns
        return self

    #Transform
    def transform(self, X):
        #Creamos una copia de X para no perder los datos originales
        Xaux=X.copy()
        #Accedemos a la columna de habilidades, y modificamos las columnas acorde
        for c in self.columns:
            caracteresAEliminar = ["[", "]", "'"]
            for char in caracteresAEliminar:
                Xaux.loc[:,c]= Xaux.loc[:,c].str.replace(char,'')
            #Para los Pokémon que tienen sólo una habilidad, vamos a añadirles dos habilidades nulas; y para los
            #que tengan 2 habilidades, les añadimos una extra.
            for poke in Xaux.loc[:,c].index:
                if countCaracter(Xaux.loc[poke,c], ",")==0:
                    Xaux.loc[poke,c]=Xaux.loc[poke,c]+", None, None"
                elif countCaracter(Xaux.loc[poke,c], ",")==1:
                    Xaux.loc[poke,c]=Xaux.loc[poke,c]+", None"
            #Creamos tres columnas nuevas
            Xaux[['Ability 1', 'Ability 2', 'Ability 3']] = Xaux.loc[:,c].str.split(',', expand=True)
            #Por último, hay que borrar la columna de "Abilities" original
            Xaux=Xaux.drop(["Abilities"], axis=1)
        return Xaux

In [ ]:
categorizadorDeHabilidades = AbilityCategorizer(columns = ["Abilities"])
categorizadorDeHabilidades.fit(X_ConTipos)
X_ConTiposHabilidades = categorizadorDeHabilidades.transform(X_ConTipos)
X_ConTiposHabilidades

Los nombres de los Pokémon no tienen relevancia competitiva, por lo que simplemente vamos a aplicar una codificación ordinal sobre esta columnna. Previsiblemente, al realizar la selección de variables, esta variable no será escogida, pero de igual forma necesitamos codificarla.

In [ ]:
encodificadorOrdinal = ce.OrdinalEncoder(cols=["Name"])
#La entrenamos con el DataFrame sin datos faltantes
encodificadorOrdinal.fit(X_ConTiposHabilidades)
#Hacemos la transformación
X_ConTiposHabilidadesNombres=encodificadorOrdinal.transform(X_ConTiposHabilidades)
X_ConTiposHabilidadesNombres

La columna de "siguientes evoluciones" puede tener relevancia ya que los Pokémon que todavía pueden evolucionar pueden utilizar un objeto llamado Mineral Evolutivo, que aumenta sus defensas en un 50%. No nos importa tanto si el Pokémon puede evolucionar una vez, dos veces o si tiene diferentes posibles evoluciones; solo si es capaz de evolucionar. Por lo tanto, codificaremos esta columna de la siguiente manera: si el Pokémon tiene alguna evolución posible, le asignaremos el valor 1; en caso contrario, le asignaremos el valor 0. Para ello, vamos a modificar la columna aplicándole dos tratamientos. Si la fila correspondiente es "[]", pondremos "´No", y en caso contrario pondremos "Si". Después, aplicaremos una codificación One Hot, adecuada ya que sólo generará una columna adicional al poder tomar esta variable solo dos valores.

Además, hay algunos Pokémon que pueden tener formas alternativas (que no se consideran evoluciones, por lo que no permiten el uso del Mineral Evolutivo) y que en el dataset presentan una lista no vacía en la columna de "siguientes evoluciones". Como no hay ningún criterio para detectar estos casos, los vamos a tratar manualmente. Estos Pokémon son **Giratina**, **Greninja**, **Kyurem**, **Landorus**, **Oricorio**, **Rotom**, **Shaymin**, **Thundurus**, **Tornadus** y **Zygarde**. Pese a que tengan una lista vacía, estos Pokémon los vamos a codificar como que **no** tienen evolución.

In [76]:
class EvolutionCategorizer(TransformerMixin):
    # Constructor de la clase
    def __init__(self, columns=None): #Si recibimos algo
        self.columns = columns

    #Fit
    def fit(self, X, y=0):
        X=pd.DataFrame(X) #Por si recibiéramos un array de numpy
        if self.columns == None:
            self.columns=X.columns
        return self

    #Transform
    def transform(self, X, y=0):
        #Creamos una copia de X para no perder los datos originales
        Xaux=X.copy()
        #Accedemos a la columna de tipos, y rellenamos las columnas acorde
        for c in self.columns:
            Xaux.loc[:,c] = np.where(Xaux.loc[:,c]=="[]", "No", "Si")
            #Ahora, consideramos las excepciones descritas arriba
            pokemonEquivocados = ["Giratina", "Greninja", "Kyurem", "Landorus", "Oricorio", "Rotom", "Shaymin", "Thundurus", "Tornadus", "Zygarde"]
            for poke in pokemonEquivocados:
                Xaux.loc[poke,c]="No"
        return Xaux

    #set_params
    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self

#Una vez aplicamos el EvolutionCategorizer(), aplicamos el One Hot Encoding. Para ello, hacemos una PipeLine
pipe_Evol =  Pipeline([('Categorizador de Evoluciones', EvolutionCategorizer(columns=["Next Evolution(s)"])), 
                       ('One Hot Encoding', ce.OneHotEncoder(cols = ["Next Evolution(s)"], handle_unknown="ignore"))])

In [ ]:
pipe_Evol.fit(X_ConTiposHabilidadesNombres)
datosTiposHabilidadesNombresEvol = pipe_Evol.transform(X_ConTiposHabilidadesNombres)
datosTiposHabilidadesNombresEvol

Dividimos los datos en: Conjunto de entrenamiento, validación y test.

In [78]:
# Creación de los conjuntos de test (10% de los ejemplos) y el resto de ejemplos
restoEjemplos_X, X_test, restoEjemplos_y, y_test =  train_test_split(X.loc[:,:],y.loc[:], test_size=0.10, random_state=123)

# Creación de los conjuntos de train (80% de los ejemplos) y de validación (resto de los ejemplos)
X_train, X_val, y_train, y_val =  train_test_split(restoEjemplos_X,restoEjemplos_y, train_size=0.80, test_size=0.20, random_state=123)

Buscamos Outliers entre los datos mediante el método del rango intercuartílico. Sin embargo, puede tener sentido no borrar los outliers debido a que las estadísticas de los Pokemons no son mediciones sino sus valores "reales".

In [79]:
# Creamos la clase de detección y tratamiento de Outliers
class OutlierDetecion_treatment_IQR(TransformerMixin):
    nombresVariablesNum = ['HP', 'Attack', 'Defense', 'Special Attack', 'Special Defense','Speed']
    # Constructor de la clase
    def __init__(self, k=1.5, columns=nombresVariablesNum):
        self.k = k
        self.columns = columns
    
    # Método fit
    def fit(self, X, y=None):
        # Transformamos X a DataFrame por si llega un array de Numpy (para compatibilidad en la Pipeline)
        X = pd.DataFrame(X)
        if self.columns == None:
            # Si no se determinan variables en el constructor si tratan todas
            self.columns = X.columns
        self.stats = X.loc[:,self.columns].describe()
        # Devolvemos el propio objeto modificado
        return self

    # Método transform
    def transform(self, X):
        # Transformamos X a DataFrame por si llega un array de Numpy (para compatibilidad en la Pipeline)
        X = pd.DataFrame(X)
        # Creamos una copia del DataFrame X para no perder los datos originales
        Xaux = X.copy()
        # Se calcula el IQR de cada variable
        IQRs = self.stats.loc['75%'] - self.stats.loc['25%']
        # Se calculan los límites inferiores y superiores   
        limiteInf = self.stats.loc['25%'] - self.k * IQRs
        limiteSup = self.stats.loc['75%'] + self.k * IQRs
        # Se comprueba qué elementos están por encima y por debajo de dichos límites (máscaras de booleanos)  
        print(limiteInf)
        print(limiteSup)
        menores = Xaux.loc[:,self.nombresVariablesNum] < limiteInf
        mayores = Xaux.loc[:,self.nombresVariablesNum] > limiteSup

        # Se recorren las variables para detectar outliers y tratarlos (sustituir por la mediana de la variable)
        for c in self.columns:
            # obtenemos la lista de booleanos correspondientes a si los valores de los ejemplos son outliers o no para la variable c
            indices = np.logical_or(menores[c] , mayores[c])
            # Si hay outliers
            if indices.any():
                # Los sustituimos por la mediana
                print("Se han encontrado outliers en la variable", c)
                print(Xaux.loc[indices, c])
                #Xaux.loc[indices, c] = self.stats.loc['50%', c]
        # Se devuelve el DataFrame modificado
        return Xaux
    
    # Método para asignar los valores de los híper-parámetros y que, de este modo, 
        # podamos aplicar GridSearchCV sobre un objeto de esta clase
    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self
    
    # Método para obtener los valores de los híper-parámetros que queramos del modelo (lo usa GridSearchCV al mostrar la mejor configuración)
    def get_params(self, deep=True):
        # Devolvemos los valores de los híper-parámetros del método de preparación de datos
        return {"k": self.k}

In [ ]:
# Instanciamos la clase anterior (llamamos out_IQR al objeto), la entrenamos con los datos de train y transformamos el conjunto de train
out_IQR = OutlierDetecion_treatment_IQR(k = 3)
out_IQR = out_IQR.fit(X_train)
X_train_IQR = out_IQR.transform(X_train)
print(X_train_IQR)
